看看均线策略。

In [ ]:
import pandas as pd
from datetime import datetime
import trdb2py

isStaticImg = False
width = 960
height = 768

pd.options.display.max_columns = None
pd.options.display.max_rows = None

trdb2cfg = trdb2py.loadConfig('./trdb2.yaml')

我们测测BTC，特定的时间段来分析吧。

In [ ]:
# 具体基金
asset = 'bitmex.XBTUSD|1d'

# 起始时间，0表示从最开始算起
tsStart = 0
# tsStart = int(trdb2py.str2timestamp('2018-03-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
tsEnd = -1
# tsEnd = int(trdb2py.str2timestamp('2019-01-01', '%Y-%m-%d'))

# 初始资金池
paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

# 买入参数，用全部的钱来买入（也就是复利）
paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

# 卖出参数，全部卖出
paramssell = trdb2py.trading2_pb2.SellParams(
    perVolume=1,
)

首先看看这个基金的基准表现，就是在开始时间就直接买入，然后一直持有，看具体的收益率。

In [ ]:
# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='baseline',
)  

pnlBaseline = trdb2py.simTrading(trdb2cfg, p0)
# trdb2py.showPNL(pnlBaseline, dtFormat='%Y-%m-%d %H:%M:%S', toImg=isStaticImg, width=width, height=height)
trdb2py.showPNL(pnlBaseline, toImg=isStaticImg, width=width, height=height)

那么策略基准线大概就是这样了。

换到日线来，BTC的变化就非常惊人了，5年的时间，120倍，其中有一次非常大的回撤

接下来，测一下均线

In [ ]:
lstparams = []

for ema in range(5, 61):    
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['upcross'],
        strVals=['ema.{}'.format(ema)],
    )
    
    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[ema],
    )    
            
    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['downcross'],
        strVals=['ema.{}'.format(ema)],
    )
    
    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0, buy1])
    s0.sell.extend([sell0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='ema{}'.format(ema),
    ))
 
lstpnlmix = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=20)

trdb2py.showPNLs(lstpnlmix + [pnlBaseline], dtFormat='%Y-%m-%d %H:%M:%S', toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport(lstpnlmix + [pnlBaseline])

dfpnl1 = dfpnl[dfpnl['totalReturns'] >= 20]

dfpnl1[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

可以看到，切换到日线以后，对于btc这种上涨资产来说，简单的单均线就能盈利了

In [ ]:
lstparams = []

for ema0 in range(5, 61):
    for ema1 in range(ema0, 61):
        buy0 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatorsp',
            operators=['upcross'],
            strVals=['ema.{}'.format(ema0)],
        )

        buy1 = trdb2py.trading2_pb2.CtrlCondition(
            name='waittostart',
            vals=[ema0],
        )
        
        buy2 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatorsp',
            operators=['up'],
            strVals=['ema.{}'.format(ema1)],
        )
        
        buy3 = trdb2py.trading2_pb2.CtrlCondition(
            name='waittostart',
            vals=[ema1],
        )        

        sell0 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatorsp',
            operators=['downcross'],
            strVals=['ema.{}'.format(ema0)],
        )

        s0 = trdb2py.trading2_pb2.Strategy(
            name="normal",
            asset=trdb2py.str2asset(asset),
        )

        s0.buy.extend([buy0, buy1, buy2, buy3])
        s0.sell.extend([sell0])
        s0.paramsBuy.CopyFrom(paramsbuy)
        s0.paramsSell.CopyFrom(paramssell) 
        s0.paramsInit.CopyFrom(paramsinit)        
        lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
            assets=[trdb2py.str2asset(asset)],
            startTs=tsStart,
            endTs=tsEnd,
            strategies=[s0],
            title='ema{}&ema{}'.format(ema0, ema1),
        ))
 
lstpnlmix = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=180)

trdb2py.showPNLs(lstpnlmix + [pnlBaseline], dtFormat='%Y-%m-%d %H:%M:%S', toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport(lstpnlmix + [pnlBaseline])

dfpnl1 = dfpnl[dfpnl['totalReturns'] >= 1]

dfpnl1[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

考虑2条均线时，几乎能到200倍回报了

就算最后换了一个下跌区间（2018-01-01 2020-03-01），其实也是明显盈利的

在小时数据里，我用过7天的均线图，也就是168小时，其实效果也非常不理想，但日线里，就算7天的均线，其实也不错了

这2者有个明显的区别就是粒度越大的数据，因为只用close数据，所以中间会过滤很多噪声，而小时数据里，就算均线粒度给大，其实还是掺杂了很多噪声，所以效果会不太一样